In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os

from jamie import JAMIE
from jamie.evaluation import *
from jamie.utilities import *
import matplotlib.pyplot as plt
from mmd_wrapper import mmd_combine
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing

2023-01-04 06:22:48.037038: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-04 06:22:48.596477: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-04 06:22:48.679366: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-04 06:22:48.679399: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
np.random.seed(42)

# MMD-MA

In [4]:
dataset_name = 'MMD-MA'
dataset_color = 'lime'
modality_names = ['Modality 1', 'Modality 2']
data_folder = '../data/UnionCom/MMD/'
data1 = np.loadtxt(data_folder + "s1_mapped1.txt")
data2 = np.loadtxt(data_folder + "s1_mapped2.txt")
type1 = np.loadtxt(data_folder + "s1_type1.txt")
type2 = np.loadtxt(data_folder + "s1_type2.txt")
type1 = type1.astype(np.int)
type2 = type2.astype(np.int)
type1 = np.array([f'Cell Type {i}' for i in type1])
type2 = np.array([f'Cell Type {i}' for i in type2])

# Labels
labels = [type1, type2]
features = [None, None]
feature_dict = {}

/tmp/ipykernel_14435/4179390009.py:9: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  type1 = type1.astype(np.int)
/tmp/ipykernel_14435/4179390009.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for 

In [5]:
# Preprocessing
data1 = preprocessing.scale(data1, axis=0)
data2 = preprocessing.scale(data2, axis=0)
data1[np.isnan(data1)] = 0  # Replace NaN with average
data2[np.isnan(data2)] = 0
# data1 = preprocessing.MinMaxScaler().fit_transform(data1)
# data2 = preprocessing.MinMaxScaler().fit_transform(data2)
dataset = [data1, data2]

# Replace NULL feature names
for i in range(len(features)):
    if features[i] is None:
        features[i] = np.array([f'Feature {i}' for i in range(dataset[i].shape[1])])
        
# Train-Test Imputation
train_size = int(.8 * len(data1))
train_idx = np.random.choice(range(len(data1)), train_size, replace=False)
test_idx = np.array(list(set(range(len(data1))) - set(train_idx)))

# Reduced Priors
full_priors = np.eye(len(dataset[0]))
random_idx = np.random.choice(range(len(dataset[0])), int(.5 * len(dataset[0])), replace=False)
priors = np.zeros(len(dataset[0]))
priors[random_idx] = 1
half_priors = np.diag(priors)
random_idx = np.random.choice(range(len(dataset[0])), int(.75 * len(dataset[0])), replace=False)
priors = np.zeros(len(dataset[0]))
priors[random_idx] = 1
tq_priors = np.diag(priors)
none_priors = np.zeros((len(dataset[0]), len(dataset[0])))

In [6]:
jm = JAMIE(epoch_DNN=100, debug=True, enable_memory_logging=True)
jm_data = jm.fit_transform(dataset=dataset)

use random seed: 666
Shape of Raw data
Dataset 0: (300, 2000)
Dataset 1: (300, 1000)
Device: cpu
---------------------------------
Find correspondence between Dataset 1 and Dataset 2
epoch:[500/2000] err:11.7152 alpha:0.1578
epoch:[1000/2000] err:10.2456 alpha:0.3746
epoch:[1500/2000] err:10.1076 alpha:0.4565
epoch:[2000/2000] err:10.0670 alpha:0.4759
Finished Matching!
---------------------------------
Train coupled autoencoders


/mnt/c/Users/nck/repos/nmacom/jamie/jamie.py:427: UserWarning: PCA dim must be lower than 300, found 512, adjusting to compensate.
  warnings.warn(
/mnt/c/Users/nck/repos/nmacom/jamie/jamie.py:427: UserWarning: PCA dim must be lower than 300, found 512, adjusting to compensate.
  warnings.warn(


KL: 0.0077  Rec: 0.3936  CosSim: 0.1246  F: 0.0076
Finished Mapping!
---------------------------------
JAMIE Done!
Distance: 0.863750461001473
Distance Memory: Stored 1777383 - Peak 2772492
Correspondence: 25.195908704998146
Correspondence Memory: Stored 372504 - Peak 1441931
Mapping: 5.137437727003999
Mapping Memory: Stored 8881502 - Peak 18051533
Total: 31.197096893003618



# Simulation 1250

In [7]:
dataset_name = 'scMultiSim-1250'
dataset_color = 'teal'
modality_names = ['Modality 1', 'Modality 2']
data_folder = '../data/scMultiSim/new/'
data1 = np.loadtxt(data_folder + "scMultiSim_RNA_counts_1250_genes.csv", delimiter=",", skiprows=1)
data2 = np.loadtxt(data_folder + "scMultiSim_ATAC_seq_1250_genes_new.csv", delimiter=",", skiprows=1)
ct = pd.read_csv(data_folder + "cell_meta_1250_genes.csv").iloc[3, 1:].to_numpy().astype(int)
type1 = type2 = np.array([f'Cell Type {i}' for i in ct])

# Labels
labels = [type1, type2]
features = [None, None]
feature_dict = {}

# Utility
positivize = lambda X: [x + x.min() for x in X]
minmax = lambda X: [(x + x.min()) for x in X]

In [8]:
# Preprocessing
data1 = preprocessing.scale(data1, axis=0)
data2 = preprocessing.scale(data2, axis=0)
data1[np.isnan(data1)] = 0  # Replace NaN with average
data2[np.isnan(data2)] = 0
# data1 = preprocessing.MinMaxScaler().fit_transform(data1)
# data2 = preprocessing.MinMaxScaler().fit_transform(data2)
dataset = [data1, data2]

# Replace NULL feature names
for i in range(len(features)):
    if features[i] is None:
        features[i] = np.array([f'Feature {i}' for i in range(dataset[i].shape[1])])
        
# Train-Test Imputation
train_size = int(.8 * len(data1))
train_idx = np.random.choice(range(len(data1)), train_size, replace=False)
test_idx = np.array(list(set(range(len(data1))) - set(train_idx)))

# Reduced Priors
full_priors = np.eye(len(dataset[0]))
random_idx = np.random.choice(range(len(dataset[0])), int(.5 * len(dataset[0])), replace=False)
priors = np.zeros(len(dataset[0]))
priors[random_idx] = 1
half_priors = np.diag(priors)
random_idx = np.random.choice(range(len(dataset[0])), int(.75 * len(dataset[0])), replace=False)
priors = np.zeros(len(dataset[0]))
priors[random_idx] = 1
tq_priors = np.diag(priors)
none_priors = np.zeros((len(dataset[0]), len(dataset[0])))

In [9]:
jm = JAMIE(epoch_DNN=100, debug=True, enable_memory_logging=True)
jm_data = jm.fit_transform(dataset=dataset)

use random seed: 666
Shape of Raw data
Dataset 0: (500, 1250)
Dataset 1: (500, 3750)
Device: cpu
---------------------------------
Find correspondence between Dataset 1 and Dataset 2
epoch:[500/2000] err:13.0043 alpha:0.0774
epoch:[1000/2000] err:16.3444 alpha:0.1053
epoch:[1500/2000] err:18.7388 alpha:0.1709
epoch:[2000/2000] err:21.0007 alpha:0.4030
Finished Matching!
---------------------------------
Train coupled autoencoders


/mnt/c/Users/nck/repos/nmacom/jamie/jamie.py:427: UserWarning: PCA dim must be lower than 500, found 512, adjusting to compensate.
  warnings.warn(
/mnt/c/Users/nck/repos/nmacom/jamie/jamie.py:427: UserWarning: PCA dim must be lower than 500, found 512, adjusting to compensate.
  warnings.warn(


KL: 0.0116  Rec: 1.1089  CosSim: 0.0667  F: 0.0054
Finished Mapping!
---------------------------------
JAMIE Done!
Distance: 0.3170076840033289
Distance Memory: Stored 4164346 - Peak 6543329
Correspondence: 16.31046741300088
Correspondence Memory: Stored 1015342 - Peak 4001195
Mapping: 15.335240792999684
Mapping Memory: Stored 24233878 - Peak 62105969
Total: 31.962715890003892



# scMNC Motor

In [10]:
dataset_name = 'scMNC-Motor'
dataset_color = 'sienna'
modality_names = ['Gene Expression', 'Electrophysiology']
data_folder = '../data/scMNC/mouse_motor_cortex/data/'
data1 = pd.read_csv(data_folder + "geneExp_filtered.csv")
data2 = pd.read_csv(data_folder + "efeature_filtered.csv")
feat1 = np.array(data1.iloc[:, 0])
feat2 = np.array(data2.columns)
sample_names1 = data1.columns[1:]
assert ((data1.shape[1] - 1) == data2.shape[0])
data1 = np.transpose(np.array(data1)[:, 1:])
data2 = np.array(data2)
meta = pd.read_excel(data_folder + "motor_meta_data.xlsx")[['Cell', 'RNA family']]
meta = np.array(meta)
meta_idx = [np.argwhere(meta[:, 0] == sample_names1[i])[0][0] for i in range(sample_names1.shape[0])]
type1 = type2 = np.array([x.split()[0] for x in meta[meta_idx, 1]])

# Sampling
# split = 1000 # data1.shape[0]
# data_col_idx = np.random.choice(range(data1.shape[0]), split, replace=False)
# data1, data2, type1, type2 = (x[data_col_idx] for x in (data1, data2, type1, type2))

# Labels
labels = [type1, type2]
features = [feat1, feat2]
feature_dict = {}

# Utility
positivize = lambda X: [x + x.min() for x in X]
minmax = lambda X: [(x + x.min()) for x in X]

In [11]:
# Preprocessing
data1 = preprocessing.scale(data1, axis=0)
data2 = preprocessing.scale(data2, axis=0)
data1[np.isnan(data1)] = 0  # Replace NaN with average
data2[np.isnan(data2)] = 0
# data1 = preprocessing.MinMaxScaler().fit_transform(data1)
# data2 = preprocessing.MinMaxScaler().fit_transform(data2)
dataset = [data1, data2]

# Replace NULL feature names
for i in range(len(features)):
    if features[i] is None:
        features[i] = np.array([f'Feature {i}' for i in range(dataset[i].shape[1])])
        
# Train-Test Imputation
train_size = int(.8 * len(data1))
train_idx = np.random.choice(range(len(data1)), train_size, replace=False)
test_idx = np.array(list(set(range(len(data1))) - set(train_idx)))

# Reduced Priors
full_priors = np.eye(len(dataset[0]))
random_idx = np.random.choice(range(len(dataset[0])), int(.5 * len(dataset[0])), replace=False)
priors = np.zeros(len(dataset[0]))
priors[random_idx] = 1
half_priors = np.diag(priors)
random_idx = np.random.choice(range(len(dataset[0])), int(.75 * len(dataset[0])), replace=False)
priors = np.zeros(len(dataset[0]))
priors[random_idx] = 1
tq_priors = np.diag(priors)
none_priors = np.zeros((len(dataset[0]), len(dataset[0])))

In [12]:
jm = JAMIE(epoch_DNN=100, debug=True, enable_memory_logging=True)
jm_data = jm.fit_transform(dataset=dataset)

use random seed: 666
Shape of Raw data
Dataset 0: (1208, 1286)
Dataset 1: (1208, 29)
Device: cpu
---------------------------------
Find correspondence between Dataset 1 and Dataset 2
epoch:[500/2000] err:0.3861 alpha:0.0050
epoch:[1000/2000] err:3.3359 alpha:0.0249
epoch:[1500/2000] err:5.7469 alpha:0.0598
epoch:[2000/2000] err:7.0837 alpha:0.0926
Finished Matching!
---------------------------------
Train coupled autoencoders


/mnt/c/Users/nck/repos/nmacom/jamie/jamie.py:427: UserWarning: PCA dim must be lower than 29, found 512, adjusting to compensate.
  warnings.warn(


KL: 0.0033  Rec: 1.7153  CosSim: 1.3200  F: 0.0222
Finished Mapping!
---------------------------------
JAMIE Done!
Distance: 1.8937601689976873
Distance Memory: Stored 23527186 - Peak 36739750
Correspondence: 172.87693743099953
Correspondence Memory: Stored 5866579 - Peak 23349419
Mapping: 11.145951522004907
Mapping Memory: Stored 10897745 - Peak 43899487
Total: 185.91664912200213



# scMNC Visual

In [13]:
dataset_name = 'scMNC-Visual'
dataset_color = 'magenta'
modality_names = ['Gene Expression', 'Electrophysiology']
data_folder = '../data/scMNC/mouse_visual_cortex/data/'
data1 = pd.read_csv(data_folder + "geneExp_filtered.csv")
data2 = pd.read_csv(data_folder + "efeature_filtered.csv")
sample_names1 = data1.columns[1:]
sample_names2 = np.array(data2)[:, 0]
feature_names1 = data1.iloc[:,0]
feature_names2 = data2.columns[3:]
assert (sample_names1 == sample_names2).all()
data1 = np.transpose(np.array(data1)[:, 1:])
data2 = np.array(data2)[:, 3:]
meta = pd.read_csv(data_folder + "20200711_patchseq_metadata_mouse.csv")[['transcriptomics_sample_id', 't_type']]
meta = np.array(meta)
meta_idx = [np.argwhere(meta[:, 0] == sample_names1[i])[0][0] for i in range(sample_names1.shape[0])]
type1 = type2 = np.array([x.split(' ')[0] for x in meta[meta_idx, 1]])

# Sampling
# split = 1000 # data1.shape[0]
# data_col_idx = np.random.choice(range(data1.shape[0]), split, replace=False)
# data1, data2, type1, type2 = (x[data_col_idx] for x in (data1, data2, type1, type2))

# Labels
labels = [type1, type2]
features = [np.array(feature_names1), np.array(feature_names2)]
feature_dict = {}

# Utility
positivize = lambda X: [x + x.min() for x in X]
minmax = lambda X: [(x + x.min()) for x in X]

In [14]:
# Preprocessing
data1 = preprocessing.scale(data1, axis=0)
data2 = preprocessing.scale(data2, axis=0)
data1[np.isnan(data1)] = 0  # Replace NaN with average
data2[np.isnan(data2)] = 0
# data1 = preprocessing.MinMaxScaler().fit_transform(data1)
# data2 = preprocessing.MinMaxScaler().fit_transform(data2)
dataset = [data1, data2]

# Replace NULL feature names
for i in range(len(features)):
    if features[i] is None:
        features[i] = np.array([f'Feature {i}' for i in range(dataset[i].shape[1])])
        
# Train-Test Imputation
train_size = int(.8 * len(data1))
train_idx = np.random.choice(range(len(data1)), train_size, replace=False)
test_idx = np.array(list(set(range(len(data1))) - set(train_idx)))

# Reduced Priors
full_priors = np.eye(len(dataset[0]))
random_idx = np.random.choice(range(len(dataset[0])), int(.5 * len(dataset[0])), replace=False)
priors = np.zeros(len(dataset[0]))
priors[random_idx] = 1
half_priors = np.diag(priors)
random_idx = np.random.choice(range(len(dataset[0])), int(.75 * len(dataset[0])), replace=False)
priors = np.zeros(len(dataset[0]))
priors[random_idx] = 1
tq_priors = np.diag(priors)
none_priors = np.zeros((len(dataset[0]), len(dataset[0])))

In [15]:
jm = JAMIE(epoch_DNN=100, debug=True, enable_memory_logging=True)
jm_data = jm.fit_transform(dataset=dataset)

use random seed: 666
Shape of Raw data
Dataset 0: (3654, 1302)
Dataset 1: (3654, 39)
Device: cpu
---------------------------------
Find correspondence between Dataset 1 and Dataset 2
epoch:[500/2000] err:0.4526 alpha:0.0207
epoch:[1000/2000] err:0.0000 alpha:0.0000
epoch:[1500/2000] err:0.0000 alpha:0.0000
epoch:[2000/2000] err:0.0000 alpha:0.0000
Finished Matching!
---------------------------------
Train coupled autoencoders


/mnt/c/Users/nck/repos/nmacom/jamie/jamie.py:427: UserWarning: PCA dim must be lower than 39, found 512, adjusting to compensate.
  warnings.warn(


KL: 0.0126  Rec: 1.3997  CosSim: 0.0915  F: 0.0023
Finished Mapping!
---------------------------------
JAMIE Done!
Distance: 39.65111083200463
Distance Memory: Stored 213777111 - Peak 334178279
Correspondence: 4496.191194003004
Correspondence Memory: Stored 53474886 - Peak 213628651
Mapping: 49.011749792000046
Mapping Memory: Stored 22045795 - Peak 235707328
Total: 4584.854054627009



# DM_rep4

In [16]:
import scanpy as sc

babel_dir = '../data/babel_data/DM_rep4/'
train = sc.read_h5ad(babel_dir + 'train_rna.h5ad')
v = train.var_names
train = train.X.toarray()
valid = sc.read_h5ad(babel_dir + 'truth_rna.h5ad').X.toarray()
data1 = np.concatenate([train, valid], axis=0)
fnames1 = np.array(v)

train = sc.read_h5ad(babel_dir + 'train_atac.h5ad')
v = train.var_names
train = train.X.toarray()
valid = sc.read_h5ad(babel_dir + 'truth_atac.h5ad').X.toarray()
data2 = np.concatenate([train, valid], axis=0)
fnames2 = np.array(v)

split = train.shape[0]

/home/thema/miniconda3/lib/python3.9/site-packages/anndata/compat/__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/home/thema/miniconda3/lib/python3.9/site-packages/anndata/compat/__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(
/home/thema/miniconda3/lib/python3.9/site-packages/anndata/compat/__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/home/thema/miniconda3/lib/python3.9/site-packages/anndata/compat/__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(


In [17]:
dataset_name = 'DM_rep4'
dataset_color = 'black'
modality_names = ['RNA', 'ATAC']

# data1 = PCA(n_components=32).fit_transform(data1)
# data2 = PCA(n_components=32).fit_transform(data2)
# fnames1 = fnames2 = None

type1 = np.array(len(data1) * ['Cell Type 0'])
type2 = np.array(len(data2) * ['Cell Type 0'])

# Sampling
# sample_num = 500
# data_col_idx = np.random.choice(range(split), sample_num, replace=False)
# data1, data2, type1, type2 = (x[list(data_col_idx) + list(range(split, len(data1)))] for x in (data1, data2, type1, type2))
# split = sample_num

# Labels
labels = [type1, type2]
features = [np.array(fnames1), np.array(fnames2)]
feature_dict = {}

# Utility
positivize = lambda X: [x + x.min() for x in X]
minmax = lambda X: [(x + x.min()) for x in X]

In [18]:
# Preprocessing
data1 = preprocessing.scale(data1, axis=0)
data2 = preprocessing.scale(data2, axis=0)
data1[np.isnan(data1)] = 0  # Replace NaN with average
data2[np.isnan(data2)] = 0
# data1 = preprocessing.MinMaxScaler().fit_transform(data1)
# data2 = preprocessing.MinMaxScaler().fit_transform(data2)
dataset = [data1, data2]

# Replace NULL feature names
for i in range(len(features)):
    if features[i] is None:
        features[i] = np.array([f'Feature {i}' for i in range(dataset[i].shape[1])])
        
# # Train-Test Imputation
train_size = split
train_idx = np.array(range(split))
test_idx = np.array(list(set(range(len(data1))) - set(train_idx)))

# Reduced Priors
full_priors = np.eye(len(dataset[0]))
random_idx = np.random.choice(range(len(dataset[0])), int(.5 * len(dataset[0])), replace=False)
priors = np.zeros(len(dataset[0]))
priors[random_idx] = 1
half_priors = np.diag(priors)
none_priors = np.zeros((len(dataset[0]), len(dataset[0])))

/home/thema/miniconda3/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:235: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/home/thema/miniconda3/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:254: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
/home/thema/miniconda3/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:235: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/home/thema/miniconda3/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:254: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. 

In [19]:
jm = JAMIE(epoch_DNN=100, debug=True, enable_memory_logging=True)
jm_data = jm.fit_transform(dataset=dataset)

use random seed: 666
Shape of Raw data
Dataset 0: (4301, 34861)
Dataset 1: (4301, 85596)
Device: cpu
---------------------------------
Find correspondence between Dataset 1 and Dataset 2
epoch:[500/2000] err:3240.7747 alpha:318.6570
epoch:[1000/2000] err:202.1322 alpha:18.4804
epoch:[1500/2000] err:0.6152 alpha:0.0076
epoch:[2000/2000] err:0.0001 alpha:0.0000
Finished Matching!
---------------------------------
Train coupled autoencoders
KL: 0.0116  Rec: 1.8302  CosSim: 0.0442  F: 0.0019
Finished Mapping!
---------------------------------
JAMIE Done!
Distance: 435.5420048619999
Distance Memory: Stored 296179081 - Peak 5410347212
Correspondence: 7009.672455397995
Correspondence Memory: Stored 74064707 - Peak 295978811
Mapping: 190.9282204590054
Mapping Memory: Stored 287755346 - Peak 3215879529
Total: 7636.142680719



# brainchromatin

In [20]:
dataset_name = 'BrainChromatin'
dataset_color = 'orange'
modality_names = ['RNA', 'ATAC']
data_folder = '../data/brainchromatin/'
data1 = pd.read_csv(data_folder + "multiome_rna_counts.tsv", delimiter='\t').transpose()
data2 = pd.read_csv(data_folder + "multiome_atac_gene_activities.tsv", delimiter='\t', nrows=35000).transpose()
data2 = data2.transpose()[data1.index].transpose()

meta = pd.read_csv(data_folder + "multiome_cell_metadata.txt", delimiter='\t')
meta_names = pd.read_csv(data_folder + "multiome_cluster_names.txt", delimiter='\t')
meta_names = meta_names[meta_names['Assay'] == 'Multiome ATAC']
meta = pd.merge(meta, meta_names, left_on='ATAC_cluster', right_on='Cluster.ID', how='left')
meta.index = meta['Cell.ID']

type1 = type2 = np.array(meta.transpose()[data1.index].transpose()['Cluster.Name'])
fname1, fname2 = data1.columns, data2.columns
data1 = data1.to_numpy()
data2 = data2.to_numpy()

# Sampling
split = 4000 # data1.shape[0]
data_row_idx = np.random.choice(range(data1.shape[0]), split, replace=False)
# data1, data2, type1, type2 = (x[data_row_idx] for x in (data1, data2, type1, type2))
# split_feat_1 = 2000 # data1.shape[1]
# data_col1_idx = np.random.choice(range(data1.shape[1]), split_feat_1, replace=False)
# data1, fname1 = data1[:, data_col1_idx], fname1[data_col1_idx]
# split_feat_2 = 2000 # data2.shape[1]
# data_col2_idx = np.random.choice(range(data2.shape[1]), split_feat_2, replace=False)
# data2, fname2 = data2[:, data_col2_idx], fname2[data_col2_idx]

# Labels
labels = [type1, type2]
features = [np.array(fname1), np.array(fname2)]
feature_dict = {'ENSG00000251562': 'MALAT1', 'ENSG00000153707': 'PTPRD'}

# Utility
positivize = lambda X: [x + x.min() for x in X]
minmax = lambda X: [(x + x.min()) for x in X]

# Change Labels
group = ['GluN3', 'GluN4', 'IN1', 'GluN2', 'IN2', 'GluN6', 'GluN5', 'RG',
       'nIPC', 'GluN1', 'mGPC/OPC', 'IN3', 'IN4', 'SP', 'GluN7',
       'MG/EC/Peric.']
conv =  ['GluN', 'GluN', 'IN', 'GluN', 'IN', 'GluN', 'GluN', 'RG',
        'nIPC', 'GluN', 'mGPC/OPC', 'IN', 'IN', 'SP', 'GluN',
        'MG/EC/Peric.']
group_conv = {g:c for g, c in zip(group, conv)}
labels = [np.array([group_conv[l] for l in label]) for label in labels]

In [21]:
# Preprocessing
data1 = preprocessing.scale(data1, axis=0)
data2 = preprocessing.scale(data2, axis=0)
data1[np.isnan(data1)] = 0  # Replace NaN with average
data2[np.isnan(data2)] = 0
# data1 = preprocessing.MinMaxScaler().fit_transform(data1)
# data2 = preprocessing.MinMaxScaler().fit_transform(data2)
dataset = [data1, data2]

# Replace NULL feature names
for i in range(len(features)):
    if features[i] is None:
        features[i] = np.array([f'Feature {i}' for i in range(dataset[i].shape[1])])
        
# Train-Test Imputation
train_size = int(.8 * len(data1))
train_idx = np.random.choice(range(len(data1)), train_size, replace=False)
test_idx = np.array(list(set(range(len(data1))) - set(train_idx)))

# Reduced Priors
full_priors = np.eye(len(dataset[0]))
random_idx = np.random.choice(range(len(dataset[0])), int(.5 * len(dataset[0])), replace=False)
priors = np.zeros(len(dataset[0]))
priors[random_idx] = 1
half_priors = np.diag(priors)
random_idx = np.random.choice(range(len(dataset[0])), int(.75 * len(dataset[0])), replace=False)
priors = np.zeros(len(dataset[0]))
priors[random_idx] = 1
tq_priors = np.diag(priors)
none_priors = np.zeros((len(dataset[0]), len(dataset[0])))

In [22]:
jm = JAMIE(epoch_DNN=100, debug=True, enable_memory_logging=True)
jm_data = jm.fit_transform(dataset=dataset)

use random seed: 666
Shape of Raw data
Dataset 0: (8981, 34104)
Dataset 1: (8981, 19836)
Device: cpu
---------------------------------
Find correspondence between Dataset 1 and Dataset 2
epoch:[500/2000] err:1774.8389 alpha:23.8913
epoch:[1000/2000] err:324.3965 alpha:0.0213
epoch:[1500/2000] err:152.1128 alpha:0.0015
epoch:[2000/2000] err:67.9232 alpha:0.0002
Finished Matching!
---------------------------------
Train coupled autoencoders
KL: 0.0116  Rec: 1.7395  CosSim: 0.0402  F: 0.0018
Finished Mapping!
---------------------------------
JAMIE Done!
Distance: 694.3321282499965
Distance Memory: Stored 1290699169 - Peak 2017201098
Correspondence: 44623.54953084101
Correspondence Memory: Stored 322791094 - Peak 1290538381
Mapping: 270.97620322300645
Mapping Memory: Stored 300885686 - Peak 5081428501
Total: 45588.85786231401



# scGLUE

In [23]:
import scanpy as sc

dataset_name = 'scGLUE'
dataset_color = 'yellow'
modality_names = ['RNA', 'ATAC']
data_folder = '../data/scGLUE/'

data1 = sc.read_h5ad(data_folder + 'Chen-2019-RNA.h5ad')
type1 = data1.obs.cell_type.to_numpy()
fname1 = data1.var.name.to_numpy()
data1 = data1.X.todense()

data2 = sc.read_h5ad(data_folder + 'Chen-2019-ATAC.h5ad')
type2 = data2.obs.cell_type.to_numpy()
fname2 = data2.var.index.to_numpy()
data2 = data2.X.todense()

# Labels
labels = [type1, type2]
features = [fname1, fname2]

In [24]:
# Preprocessing
data1 = preprocessing.scale(data1, axis=0)
data2 = preprocessing.scale(data2, axis=0)
data1[np.isnan(data1)] = 0  # Replace NaN with average
data2[np.isnan(data2)] = 0
# data1 = preprocessing.MinMaxScaler().fit_transform(data1)
# data2 = preprocessing.MinMaxScaler().fit_transform(data2)
dataset = [data1, data2]

# Replace NULL feature names
for i in range(len(features)):
    if features[i] is None:
        features[i] = np.array([f'Feature {i}' for i in range(dataset[i].shape[1])])
        
# Train-Test Imputation
train_size = int(.8 * len(data1))
train_idx = np.random.choice(range(len(data1)), train_size, replace=False)
test_idx = np.array(list(set(range(len(data1))) - set(train_idx)))

# Reduced Priors
full_priors = np.eye(len(dataset[0]))
random_idx = np.random.choice(range(len(dataset[0])), int(.5 * len(dataset[0])), replace=False)
priors = np.zeros(len(dataset[0]))
priors[random_idx] = 1
half_priors = np.diag(priors)
random_idx = np.random.choice(range(len(dataset[0])), int(.75 * len(dataset[0])), replace=False)
priors = np.zeros(len(dataset[0]))
priors[random_idx] = 1
tq_priors = np.diag(priors)
none_priors = np.zeros((len(dataset[0]), len(dataset[0])))

/home/thema/miniconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:585: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/home/thema/miniconda3/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:235: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/home/thema/miniconda3/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:254: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
/home/thema/miniconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:585: FutureWarning: np.matrix usage is deprecated 

In [25]:
jm = JAMIE(epoch_DNN=100, debug=True, enable_memory_logging=True, loss_weights=[1,1,1,0], use_f_tilde=False)
jm_data = jm.fit_transform(dataset=dataset)

use random seed: 666
Shape of Raw data
Dataset 0: (9190, 28930)
Dataset 1: (9190, 241757)
---------------------------------
Train coupled autoencoders
Epoch: 100 - KL: 0.0130  Rec: 1.7942  CosSim: 0.0249  F: 0.0000
Finished Mapping!
---------------------------------
JAMIE Done!
Distance: 0.0038958429940976202
Distance Memory: Stored 7680 - Peak 12284
Correspondence: 2.2423492219968466
Correspondence Memory: Stored 1351445975 - Peak 1351445991
Mapping: 692.8370567480015
Mapping Memory: Stored 642474470 - Peak 18380613287
Total: 695.0833018129924



In [8]:
# jm = JAMIE(debug=True, enable_memory_logging=True)
# jm_data = jm.fit_transform(dataset=dataset)

use random seed: 666
Shape of Raw data
Dataset 0: (9190, 28930)
Dataset 1: (9190, 241757)
Device: cpu
---------------------------------
Find correspondence between Dataset 1 and Dataset 2
epoch:[500/2000] err:31392.3984 alpha:3819.6143
epoch:[1000/2000] err:8624.8457 alpha:1050.6832
epoch:[1500/2000] err:2395.9426 alpha:291.6512
epoch:[2000/2000] err:198.7031 alpha:16.8099
Finished Matching!
---------------------------------
Train coupled autoencoders
Epoch: 100 - KL: 0.0032  Rec: 0.5895  CosSim: 0.0396  F: 0.0054
Epoch: 200 - KL: 0.0059  Rec: 0.4775  CosSim: 0.0296  F: 0.0055
Epoch: 300 - KL: 0.0069  Rec: 0.3571  CosSim: 0.0194  F: 0.0047
Epoch: 400 - KL: 0.0098  Rec: 0.3073  CosSim: 0.0146  F: 0.0041
Epoch: 500 - KL: 0.0141  Rec: 0.2582  CosSim: 0.0149  F: 0.0037
epoch:[500/10000]: loss:0.305221
Epoch: 600 - KL: 0.0224  Rec: 0.2575  CosSim: 0.0092  F: 0.0027
Epoch: 700 - KL: 0.0287  Rec: 0.2464  CosSim: 0.0091  F: 0.0028
Epoch: 800 - KL: 0.0362  Rec: 0.2370  CosSim: 0.0085  F: 0.0024